In [ ]:
pip install pmdarima

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import json
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# FUNCTION: Training and testing the model on the full dataset
def predict(commodity, csv_path, json_path, p, d, q):
    print(f'You have chosen {commodity}.')

    df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
    
    # Re-train the model on the entire dataset, then predict future prices
    model = ARIMA(df['High'], order=(p, d, q))
    model = model.fit(method_kwargs={'warn_convergence': False})

    # For Future Dates
    # Set the start date to the last date recorded in CSV file
    index_future_dates = pd.date_range(start='2023-12-29', end='2024-02-28')
    
    # Run the prediction
    start = len(df)
    end = len(df) + len(index_future_dates) - 1
    predict = model.predict(start=start, end=end, typ='levels').rename('ARIMA')
    
    # Transform predict from Series to DataFrame
    predict_df = pd.DataFrame(predict)
    
    # Reset the index to turn it into a column
    predict_df = predict_df.reset_index(drop=True)
    
    # Add the future dates as a new column 'Index'
    predict_df['Index'] = index_future_dates

    # Reset the index of the column
    df = df.reset_index()
    df = df.rename(columns={'index': 'Date'})

    # Given timestamp
    start_date = df['Date'].iloc[-1]
    start_date = pd.Timestamp(start_date)
    
    # End date, which is 7 days after the start date
    end_date = start_date + pd.Timedelta(days=7)
    
    # Create a date range
    date_range = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')

    p7p = []
    p7d = []
    for day in range(7):
        filtered_df = predict_df[predict_df['Index'] == date_range[day+1]]
        p7p.append(filtered_df['ARIMA'].values[0])
        p7d.append(str(date_range[day+1]))

    # Return the results as a dictionary and convert to JSON
    results = {
        'dates': p7d,
        'prices': p7p
    }

    # Write the results to a JSON file
    with open(json_path, 'w') as f:
        json.dump(results, f)

    return results

In [ ]:
# FUNCTION: Training and testing the model on the full dataset
def predict(commodity, csv_path, json_path, p, d, q):
    print(f'You have chosen {commodity}.')

    df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
    
    # Re-train the model on the entire dataset for 'High'
    model_high = ARIMA(df['High'], order=(p, d, q))
    model_high = model_high.fit(method_kwargs={'warn_convergence': False})

    # For Future Dates
    index_future_dates = pd.date_range(start='2023-12-29', end='2024-02-28')
    
    # Run the prediction for 'High'
    start = len(df)
    end = len(df) + len(index_future_dates) - 1
    predict_high = model_high.predict(start=start, end=end, typ='levels').rename('High_ARIMA')
    
    # Transform predict from Series to DataFrame
    predict_df = pd.DataFrame(predict_high)
    
    # Reset the index to turn it into a column
    predict_df = predict_df.reset_index(drop=True)
    
    # Add the future dates as a new column 'Index'
    predict_df['Index'] = index_future_dates

    # Re-train the model on the entire dataset for 'Low'
    model_low = ARIMA(df['Low'], order=(p, d, q))
    model_low = model_low.fit(method_kwargs={'warn_convergence': False})

    # Run the prediction for 'Low'
    predict_low = model_low.predict(start=start, end=end, typ='levels').rename('Low_ARIMA')
    
    # Add 'Low' predictions to the DataFrame
    predict_df['Low_ARIMA'] = predict_low.values

    # Calculate the average and create the 'Average' column
    predict_df['Average'] = (predict_df['High_ARIMA'] + predict_df['Low_ARIMA']) / 2

    # Reset the index of the original dataframe
    df = df.reset_index()
    df = df.rename(columns={'index': 'Date'})

    # Given timestamp
    start_date = df['Date'].iloc[-1]
    start_date = pd.Timestamp(start_date)
    
    # End date, which is 7 days after the start date
    end_date = start_date + pd.Timedelta(days=7)
    
    # Create a date range
    date_range = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')

    p7p_high = []
    p7p_low = []
    p7p_avg = []
    p7d = []
    for day in range(7):
        filtered = predict_df[predict_df['Index'] == date_range[day + 1]]
        
        p7p_high.append(filtered['High_ARIMA'].values[0])
        p7p_low.append(filtered['Low_ARIMA'].values[0])
        p7p_avg.append(filtered['Average'].values[0])
        p7d.append(str(date_range[day + 1]))

    # Return the results as a dictionary and convert to JSON
    results = {
        'dates': p7d,
        'high_prices': p7p_high,
        'low_prices': p7p_low,
        'average_prices': p7p_avg
    }

    # Write the results to a JSON file
    with open(json_path, 'w') as f:
        json.dump(results, f)

    return results

In [ ]:
# NEW
# FUNCTION: Training and testing the model on the full dataset
def predict(commodity, csv_path, json_path, pdq_low, pdq_high):
    print(f'You have chosen {commodity}.')
    
    # Read the CSV file and set 'Date' as the index
    df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
    
    # Re-train the model on the entire dataset for 'High'
    model_high = ARIMA(df['High'], order=pdq_high)
    model_high = model_high.fit(method_kwargs={'warn_convergence': False})
    
    # For Future Dates
    index_future_dates = pd.date_range(start='2023-12-29', end='2024-02-28')
    
    # Run the prediction for 'High'
    start = len(df)
    end = len(df) + len(index_future_dates) - 1
    predict_high = model_high.predict(start=start, end=end, typ='levels').rename('High_ARIMA')
    
    # Transform predict from Series to DataFrame
    predict_df = pd.DataFrame(predict_high).reset_index(drop=True)
    predict_df['Index'] = index_future_dates  # Add future dates as a new column 'Index'
    
    # Re-train the model on the entire dataset for 'Low'
    model_low = ARIMA(df['Low'], order=pdq_low)
    model_low = model_low.fit(method_kwargs={'warn_convergence': False})
    
    # Run the prediction for 'Low'
    predict_low = model_low.predict(start=start, end=end, typ='levels').rename('Low_ARIMA')
    predict_df['Low_ARIMA'] = predict_low.values  # Add 'Low' predictions to DataFrame
    
    # Calculate the average and create the 'Average' column
    predict_df['Average'] = (predict_df['High_ARIMA'] + predict_df['Low_ARIMA']) / 2
    
    # Reset the index of the original dataframe
    df = df.reset_index()
    
    # Extract historical data for 'Date', 'High', and 'Low'
    historical_dates = df['Date'].dt.strftime('%Y-%m-%d').tolist()
    historical_high_prices = df['High'].tolist()
    historical_low_prices = df['Low'].tolist()
    
    # Add historical data to the results dictionary
    results = {
        'historical_dates': historical_dates,
        'historical_high_prices': historical_high_prices,
        'historical_low_prices': historical_low_prices
    }
    
    # For the 7-day prediction
    start_date = df['Date'].iloc[-1]
    start_date = pd.Timestamp(start_date)
    end_date = start_date + pd.Timedelta(days=7)
    date_range = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
    
    p7p_high, p7p_low, p7p_avg, p7d = [], [], [], []
    
    for day in range(7):
        filtered = predict_df[predict_df['Index'] == date_range[day + 1]]
        p7p_high.append(filtered['High_ARIMA'].values[0])
        p7p_low.append(filtered['Low_ARIMA'].values[0])
        p7p_avg.append(filtered['Average'].values[0])
        p7d.append(str(date_range[day + 1]))

    # Add 7-day predictions to the results dictionary
    results.update({
        'dates': p7d,
        'high_prices': p7p_high,
        'low_prices': p7p_low,
        'average_prices': p7p_avg
    })
    
    # Write the results to a JSON file
    with open(json_path, 'w') as f:
        json.dump(results, f)
    
    return results

In [ ]:
# Map the elements from the lists to their corresponding numbers
commodity = {
    'rice': {
        'regular_milled_rice': [(10, 1, 28), (21, 1, 28)],  # Low, High
        'well_milled_rice': [(14, 1, 28), (3, 1, 28)],
        'premium_rice': [(7, 1, 28), (15, 1, 28)],
        'special_rice': [(3, 0, 28), (16, 1, 28)]
    },
    'meat': {
        'beef_brisket': [(14, 1, 28), (11, 0, 15)],
        'beef_rump': [(27, 1, 28), (24, 1, 28)],
        'whole_chicken': [(15, 1, 28), (3, 0, 28)],
        'pork_belly': [(13, 1, 28), (3, 1, 19)],
        'pork_ham': [(11, 1, 28), (12, 1, 28)]
    },
    'fish': {
        'alumahan': [(11, 1, 28), (23, 1, 22)],
        'bangus': [(3, 1, 28), (2, 1, 28)],
        'galunggong': [(24, 1, 23), (13, 0, 25)],
        'tilapia': [(26, 0, 25), (2, 0, 26)]
    },
    'fruits': {
        'banana_lakatan': [(28, 1, 28), (21, 1, 28)],
        'calamansi': [(10, 1, 18), (15, 1, 21)],
        'mango': [(13, 1, 23), (9, 1, 28)],
        'papaya': [(18, 0, 28), (8, 0, 28)]
    },
    'vegetables': {
        'cabbage': [(17, 1, 24), (24, 1, 28)],
        'carrots': [(17, 1, 23), (4, 1, 24)],
        'eggplant': [(9, 0, 10), (8, 0, 13)],
        'tomato': [(25, 1, 20), (13, 1, 21)],
        'potato': [(4, 1, 28), (6, 1, 28)]
    },
    'spice': {
        'garlic': [(3, 1, 26), (4, 1, 28)],
        'onion': [(15, 0, 22), (11, 0, 18)]
    }
}

# Use the updated dictionary to pass two sets of pdq parameters
for group_commodity, specific_commodities in commodity.items():
    for specific_commodity, pdq_values in specific_commodities.items():
        pdq_low, pdq_high = pdq_values
        csv_path = '../csv/' + group_commodity + '/' + specific_commodity + '.csv'
        json_path = '../json/' + specific_commodity + '.json'
        print(csv_path)
        print(json_path)
        
        # Generate JSON files with separate pdq parameters for 'Low' and 'High'
        result = predict(specific_commodity, csv_path, json_path, pdq_low, pdq_high)
        print(result)

In [ ]:
import pandas as pd
import json
from statsmodels.tsa.arima.model import ARIMA
import random

# Function to compute open and close prices
def compute_open_close_prices(dates, high_prices, low_prices, average_prices):
    open_prices = []
    close_prices = []
    for high, low, avg in zip(high_prices, low_prices, average_prices):
        # Option 1: Based on average prices
        open_price = avg
        close_price = avg + (high - low) * 0.1  # Closing price slightly higher than average

        # Append computed values
        open_prices.append(open_price)
        close_prices.append(close_price)
    return open_prices, close_prices

# Define function to predict commodity prices and save results as JSON
def predict(commodity, csv_path, json_path, pdq_low, pdq_high):
    print(f'You have chosen {commodity}.')
    
    # Read the CSV file and set 'Date' as the index
    df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
    
    # Find the last date in the dataset
    last_date = df.index[-1]
    
    # Set the future date range to be 14 days after the last date
    end_date = last_date + pd.Timedelta(days=30)
    index_future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), end=end_date)
    
    # Re-train the model on the entire dataset for 'High'
    model_high = ARIMA(df['High'], order=pdq_high)
    model_high = model_high.fit(method_kwargs={'warn_convergence': False})
    
    # Run the prediction for 'High'
    start = len(df)
    end = len(df) + len(index_future_dates) - 1
    predict_high = model_high.predict(start=start, end=end, typ='levels').rename('High')
    
    # Re-train the model on the entire dataset for 'Low'
    model_low = ARIMA(df['Low'], order=pdq_low)
    model_low = model_low.fit(method_kwargs={'warn_convergence': False})
    
    # Run the prediction for 'Low'
    predict_low = model_low.predict(start=start, end=end, typ='levels').rename('Low')
    
    # Create a DataFrame for predictions with future dates as the index
    future_df = pd.DataFrame({
        'Date': index_future_dates,
        'High': predict_high.values,
        'Low': predict_low.values
    })
    future_df.set_index('Date', inplace=True)
    
    # Calculate the average for future data
    future_df['Average'] = (future_df['High'] + future_df['Low']) / 2
    
    # Append future data to the original data
    combined_df = pd.concat([df, future_df])
    
    # Reset index for combined data and prepare results for JSON output
    combined_df = combined_df.reset_index()
    combined_df['Date'] = combined_df['Date'].dt.strftime('%Y-%m-%d')  # Format dates for JSON compatibility
    
    # Replace NaN values in 'Average' column with None (null for JSON)
    combined_df['Average'] = combined_df['Average'].fillna(0)
    
    # Compute open and close prices
    open_prices, close_prices = compute_open_close_prices(
        combined_df['Date'], combined_df['High'], combined_df['Low'], combined_df['Average']
    )
    
    # Add open and close prices to the combined DataFrame
    combined_df['Open'] = open_prices
    combined_df['Close'] = close_prices
    
    # Convert combined data to dictionary format
    results = {
        'dates': combined_df['Date'].tolist(),
        'high_prices': combined_df['High'].tolist(),
        'low_prices': combined_df['Low'].tolist(),
        'average_prices': combined_df['Average'].tolist(),
        'open_prices': combined_df['Open'].tolist(),
        'close_prices': combined_df['Close'].tolist()
    }
    
    # Write the results to a JSON file
    with open(json_path, 'w') as f:
        json.dump(results, f)
    
    return results


# Dictionary setup and loop to call the predict function
commodity = {
    'rice': {
        'regular_milled_rice': [(10, 1, 28), (21, 1, 28)],  # Low, High
        'well_milled_rice': [(14, 1, 28), (3, 1, 28)],
        'premium_rice': [(7, 1, 28), (15, 1, 28)],
        'special_rice': [(3, 0, 28), (16, 1, 28)]
    },
    'meat': {
        'beef_brisket': [(14, 1, 28), (11, 0, 15)],
        'beef_rump': [(27, 1, 28), (24, 1, 28)],
        'whole_chicken': [(15, 1, 28), (3, 0, 28)],
        'pork_belly': [(13, 1, 28), (3, 1, 19)],
        'pork_kasim': [(11, 1, 28), (12, 1, 28)]
    },
    'fish': {
        'alumahan': [(11, 1, 28), (23, 1, 22)],
        'bangus': [(3, 1, 28), (2, 1, 28)],
        'galunggong': [(24, 1, 23), (13, 0, 25)],
        'tilapia': [(26, 0, 25), (2, 0, 26)]
    },
    'fruits': {
        'banana_lakatan': [(28, 1, 28), (21, 1, 28)],
        'calamansi': [(10, 1, 18), (15, 1, 21)],
        'mango': [(13, 1, 23), (9, 1, 28)],
        'papaya': [(18, 0, 28), (8, 0, 28)]
    },
    'vegetables': {
        'cabbage': [(17, 1, 24), (24, 1, 28)],
        'carrots': [(17, 1, 23), (4, 1, 24)],
        'eggplant': [(9, 0, 10), (8, 0, 13)],
        'tomato': [(25, 1, 20), (13, 1, 21)],
        'white_potato': [(4, 1, 28), (6, 1, 28)]
    },
    'spices': {
        'garlic': [(3, 1, 26), (4, 1, 28)],
        'red_onion': [(15, 0, 22), (11, 0, 18)]
    }
}

# Use the updated dictionary to pass two sets of pdq parameters
for group_commodity, specific_commodities in commodity.items():
    for specific_commodity, pdq_values in specific_commodities.items():
        pdq_low, pdq_high = pdq_values
        csv_path = f'../csv/{group_commodity}/{specific_commodity}.csv'
        json_path = f'../json/{specific_commodity}.json'
        
        # Generate JSON files with separate pdq parameters for 'Low' and 'High'
        result = predict(specific_commodity, csv_path, json_path, pdq_low, pdq_high)
        print(result)

# FINAL CODE

In [2]:
import warnings
import time  # Import for runtime measurement
warnings.filterwarnings('ignore')

import pandas as pd
import json
from statsmodels.tsa.arima.model import ARIMA

# Define function to predict commodity prices and save results as JSON
def predict(commodity, csv_path, json_path, pdq_low, pdq_high):
    start_time = time.time()  # Start timing the function
    print(f'You have chosen {commodity}.')
    
    try:
        # Read the CSV file and set 'Date' as the index
        df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
        
        # Find the last date in the dataset
        last_date = df.index[-1]
        
        # Set the future date range to be 31 days after the last date
        end_date = last_date + pd.Timedelta(days=31)
        index_future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), end=end_date)
        
        # Re-train the model on the entire dataset for 'High'
        model_high = ARIMA(df['High'], order=pdq_high)
        model_high = model_high.fit(method_kwargs={'warn_convergence': False})
        
        # Run the prediction for 'High'
        start = len(df)
        end = len(df) + len(index_future_dates) - 1
        predict_high = model_high.predict(start=start, end=end, typ='levels').rename('High')
        
        # Re-train the model on the entire dataset for 'Low'
        model_low = ARIMA(df['Low'], order=pdq_low)
        model_low = model_low.fit(method_kwargs={'warn_convergence': False})
        
        # Run the prediction for 'Low'
        predict_low = model_low.predict(start=start, end=end, typ='levels').rename('Low')
        
        # Create a DataFrame for predictions with future dates as the index
        future_df = pd.DataFrame({
            'Date': index_future_dates,
            'High': predict_high.values,
            'Low': predict_low.values
        })
        future_df.set_index('Date', inplace=True)
        
        # Calculate the average for future data
        future_df['Average'] = (future_df['High'] + future_df['Low']) / 2
        
        # Append future data to the original data
        combined_df = pd.concat([df, future_df])
        
        # Reset index for combined data and prepare results for JSON output
        combined_df = combined_df.reset_index()
        combined_df['Date'] = combined_df['Date'].dt.strftime('%Y-%m-%d')  # Format dates for JSON compatibility
        
        # Replace NaN values in 'Average' column with None (null for JSON)
        combined_df['Average'] = combined_df['Average'].fillna(0)
        
        # Convert combined data to dictionary format
        results = {
            'dates': combined_df['Date'].tolist(),
            'high_prices': combined_df['High'].tolist(),
            'low_prices': combined_df['Low'].tolist(),
            'average_prices': combined_df['Average'].tolist()
        }
        
        # Write the results to a JSON file
        with open(json_path, 'w') as f:
            json.dump(results, f)
        
        end_time = time.time()  # End timing the function
        print(f"Processing {commodity} completed in {end_time - start_time:.2f} seconds.")
        return results
    
    except Exception as e:
        print(f"Error processing {commodity}: {e}")
        return {}


# Dictionary setup and loop to call the predict function
commodity = {
    'rice': {
        'regular_milled_rice': [(10, 1, 28), (21, 1, 28)],  # Low, High
        'well_milled_rice': [(14, 1, 28), (3, 1, 28)],
        'premium_rice': [(7, 1, 28), (15, 1, 28)],
        'special_rice': [(3, 0, 28), (16, 1, 28)]
    },
    'meat': {
        'beef_brisket': [(14, 1, 28), (11, 0, 15)],
        'beef_rump': [(27, 1, 28), (24, 1, 28)],
        'whole_chicken': [(15, 1, 28), (3, 0, 28)],
        'pork_belly': [(13, 1, 28), (3, 1, 19)],
        'pork_kasim': [(11, 1, 28), (12, 1, 28)]
    },
    'fish': {
        'alumahan': [(11, 1, 28), (23, 1, 22)],
        'bangus': [(3, 1, 28), (2, 1, 28)],
        'galunggong': [(24, 1, 23), (13, 0, 25)],
        'tilapia': [(26, 0, 25), (2, 0, 26)]
    },
    'fruits': {
        'banana_lakatan': [(28, 1, 28), (21, 1, 28)],
        'calamansi': [(10, 1, 18), (15, 1, 21)],
        'mango': [(13, 1, 23), (9, 1, 28)],
        'papaya': [(18, 0, 28), (8, 0, 28)]
    },
    'vegetables': {
        'cabbage': [(17, 1, 24), (24, 1, 28)],
        'carrots': [(17, 1, 23), (4, 1, 24)],
        'eggplant': [(9, 0, 10), (8, 0, 13)],
        'tomato': [(25, 1, 20), (13, 1, 21)],
        'white_potato': [(4, 1, 28), (6, 1, 28)]
    },
    'spices': {
        'garlic': [(3, 1, 26), (4, 1, 28)],
        'red_onion': [(15, 0, 22), (11, 0, 18)]
    }
}

# Use the updated dictionary to pass two sets of pdq parameters
for group_commodity, specific_commodities in commodity.items():
    for specific_commodity, pdq_values in specific_commodities.items():
        pdq_low, pdq_high = pdq_values
        csv_path = f'../csv/{group_commodity}/{specific_commodity}.csv'
        json_path = f'../json/{specific_commodity}.json'
        
        # Generate JSON files with separate pdq parameters for 'Low' and 'High'
        result = predict(specific_commodity, csv_path, json_path, pdq_low, pdq_high)
        print(result)

You have chosen regular_milled_rice.
Processing regular_milled_rice completed in 23.03 seconds.
{'dates': ['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03', '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10', '2023-03-13', '2023-03-14', '2023-03-15', '2023-03-16', '2023-03-17', '2023-03-20', '2023-03-21', '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30

# CODE: WITH DIVIDE AND CONQUER

In [1]:
import warnings
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import json
from statsmodels.tsa.arima.model import ARIMA
import time

warnings.filterwarnings('ignore')


def predict(commodity, csv_path, json_path, pdq_low, pdq_high):
    start_time = time.time()
    print(f'Processing: {commodity}')
    
    try:
        df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
        
        last_date = df.index[-1]
        
        end_date = last_date + pd.Timedelta(days=31)
        index_future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), end=end_date)
        
        model_high = ARIMA(df['High'], order=pdq_high).fit(method_kwargs={'warn_convergence': False})
        predict_high = model_high.predict(
            start=len(df), end=len(df) + len(index_future_dates) - 1, typ='levels'
        ).rename('High')
        
        model_low = ARIMA(df['Low'], order=pdq_low).fit(method_kwargs={'warn_convergence': False})
        predict_low = model_low.predict(
            start=len(df), end=len(df) + len(index_future_dates) - 1, typ='levels'
        ).rename('Low')
        
        future_df = pd.DataFrame({
            'Date': index_future_dates,
            'High': predict_high.values,
            'Low': predict_low.values
        }).set_index('Date')
        future_df['Average'] = (future_df['High'] + future_df['Low']) / 2
        
        combined_df = pd.concat([df, future_df]).reset_index()
        combined_df['Date'] = combined_df['Date'].dt.strftime('%Y-%m-%d')
        combined_df['Average'] = combined_df['Average'].fillna(0)
        
        results = {
            'dates': combined_df['Date'].tolist(),
            'high_prices': combined_df['High'].tolist(),
            'low_prices': combined_df['Low'].tolist(),
            'average_prices': combined_df['Average'].tolist()
        }
        
        with open(json_path, 'w') as f:
            json.dump(results, f)
        
        end_time = time.time()
        print(f"Processing {commodity} completed in {end_time - start_time:.2f} seconds.")
        return results
    
    except Exception as e:
        print(f"Error processing {commodity}: {e}")
        return {}


def process_commodities(commodities):
    start_time = time.time()
    if len(commodities) == 1:
        group, specific_commodities = list(commodities.items())[0]
        results = []
        for specific_commodity, pdq_values in specific_commodities.items():
            for pdq in pdq_values:
                pdq_low = (pdq[0], pdq[1], pdq[2])
                pdq_high = (pdq[0], pdq[1], pdq[2])
        
                csv_path = f'../csv/{group}/{specific_commodity}.csv'
                json_path = f'../json/{specific_commodity}.json'
                result = predict(specific_commodity, csv_path, json_path, pdq_low, pdq_high)
                results.append(result)
        end_time = time.time()
        print(f"Processing group completed in {end_time - start_time:.2f} seconds.")
        return results

    mid = len(commodities) // 2
    left_part = dict(list(commodities.items())[:mid])
    right_part = dict(list(commodities.items())[mid:])

    left_results = process_commodities(left_part)
    right_results = process_commodities(right_part)

    return left_results + right_results


def process_all_groups(groups):
    start_time = time.time()
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(process_commodities, {group: commodities})
            for group, commodities in groups.items()
        ]
        for future in futures:
            results.extend(future.result())
    end_time = time.time()
    print(f"Processing all groups completed in {end_time - start_time:.2f} seconds.")
    return results


commodity_groups = {
    'rice': {
        'regular_milled_rice': [(25, 1, 0), (22, 0, 19)],
        'well_milled_rice': [(24, 0, 4), (27, 0, 2)],
        'premium_rice': [(6, 1, 10), (7, 1, 27)],
        'special_rice': [(0, 0, 1), (15, 1, 13)]
    },
    'meat': {
        'beef_brisket': [(28, 0, 33), (30, 1, 0)],
        'beef_rump': [(4, 1, 25), (27, 1, 1)],
        'whole_chicken': [(15, 1, 28), (1, 0, 0)],
        'pork_belly': [(45, 0, 37), (35, 0, 0)],
        'pork_kasim': [(39, 0, 0), (33, 0, 29)]
    },
    'fish': {
        'alumahan': [(28, 0, 33), (42, 1, 28)],
        'bangus': [(33, 1, 38), (33, 1, 45)],
        'galunggong': [(28, 0, 0), (28, 1, 0))],
        'tilapia': [(43, 0, 39), (29, 1, 0)]
    },
    'fruits': {
        'banana_lakatan': [(7, 0, 10), (15, 1, 17)],
        'calamansi': [(29, 0, 36), (45, 0, 32)],
        'mango': [(47, 2, 0), (49, 0, 0)],
        'papaya': [(31, 1, 34), (42, 1, 0)]
    },
    'vegetables': {
        'cabbage': [(49, 0, 0), (49, 0, 49)],
        'carrots': [(31, 0, 30), (40, 0, 0)],
        'eggplant': [(31, 1, 39), (35, 0, 0)],
        'tomato': [(0, 0, 15), (40, 0, 0)],
        'white_potato': [(47, 1, 32), (28, 2, 32)]
    },
    'spices': {
        'garlic': [(29, 1, 30), (31, 2, 0)],
        'red_onion': [(1, 1, 27), (5, 1, 11)]
    }
}

results = process_all_groups(commodity_groups)
print("Processing completed.")

Processing: regular_milled_rice
Processing: beef_brisket
Processing: alumahan
Processing: banana_lakatan
Processing: cabbage
Processing: garlic
Processing garlic completed in 172.42 seconds.
Processing: garlic
Processing alumahan completed in 221.49 seconds.
Processing: alumahan
Processing cabbage completed in 267.75 seconds.
Processing: cabbage
Processing beef_brisket completed in 280.13 seconds.
Processing: beef_brisket
Processing regular_milled_rice completed in 289.82 seconds.
Processing: regular_milled_rice
Processing banana_lakatan completed in 376.41 seconds.
Processing: banana_lakatan
Processing garlic completed in 299.83 seconds.
Processing: red_onion
Processing beef_brisket completed in 227.16 seconds.
Processing: beef_rump
Processing alumahan completed in 337.41 seconds.
Processing: bangus
Processing cabbage completed in 455.87 seconds.
Processing: carrots
Processing regular_milled_rice completed in 452.97 seconds.
Processing: well_milled_rice
Processing banana_lakatan compl